In [1]:
pwd

'/home/is/akiyoshi-n/my-project/notebooks/data_extraction'

### データ抽出

In [2]:
from pathlib import Path

In [3]:
DATASET_PATH = '/home/is/akiyoshi-n/my-project/data'

In [47]:
# 作成するファイル名定義
file_name = 'add_data_sub.txt'
# 使用するデータセット名
data1 = DATASET_PATH + '/DATA5000/dir001_a-a/a_d_m_a_n.txt'
data2 = DATASET_PATH + '/DATA5000/dir001_a-a/a_kanasouken.txt'
data3 = DATASET_PATH + '/DATA5000/dir001_a-a/A_S_A_P.txt'
data4 = DATASET_PATH + '/DATA5000/dir001_a-a/a_to_z.txt'
# 条件にマッチしたデータを抽出
# Python変数をシェルコマンド内で使用する時は{}で囲むがコマンドの終了の印>の後は$の後に変数名を書くと参照される
# 条件にマッチしたデータを抽出（3列目が10文字以上かつ'http'を含まない）
# 最初の5000行を取得
!awk -F'\t' 'length($3) >= 10 && !($3 ~ /http/) && !($3 ~ /@/)' {data1} {data2} {data3} {data4} > $file_name
# !cat {DATASET_PATH}/DATA5000/dir00*/* | grep -E '^.{5,}$' > {file_name}
# !cat {DATASET_PATH}/DATA5000/dir00*/* | grep 看護 > {file_name}

In [48]:
# 抽出したデータを分析
# データ数をチェック
# shellのwc(word count)コマンドを使用
# -1は行数を出力．行数 ファイル名という出力になってしまうため
# awkコマンドを使用．awkコマンドはテキストデータを使用してレポートを出力
# タイトル一つ目の空白までのものを出力という流れになっている．
!wc -l add_data_sub.txt | awk '{print $1}'

6887


In [49]:
# ファイルの最初の数行を表示
!head -n 50 add_data_sub.txt

5388869666	a_d_m_a_n	ディズニーランドホテルなう。	2009-11-03 20:02:20
5389700374	a_d_m_a_n	舞浜地ビールなう。奈緒ちゃんの粋な計らい。	2009-11-03 21:03:09
5394370590	a_d_m_a_n	ディズニーランドホテルは仕掛けがいろいろあって面白い。喫煙所が一つしかないけど…	2009-11-04 00:55:03
5394459718	a_d_m_a_n	喫煙所は完全に隔離され、ちゃちじゃないがホテル全体のトンマナに悪影響を及ぼさない設計	2009-11-04 00:58:40
5397779328	a_d_m_a_n	レヴィ＝ストロース氏死去､ 残念だ。ご冥福を祈ります。	2009-11-04 03:18:23
5397829341	a_d_m_a_n	相反する二つの目的を同時に達成するようなルール作り。これはクリエイティブ。サッカーにおけるオフサイドのようなやつね。	2009-11-04 03:20:32
5410735343	a_d_m_a_n	やべ。いい企画おもいついったー…	2009-11-04 12:20:07
5412475096	a_d_m_a_n	バズマン、その調子だ。	2009-11-04 13:41:33
5412516271	a_d_m_a_n	ディズニーのサービスクオリティって、アタマから安心できるよね。	2009-11-04 13:43:46
5412608914	a_d_m_a_n	昨日、昔のプロフェッショナル仕事の流儀がやってて、DNAのﾅﾝﾊﾞ社長が｢仕事が人を育てる｣と頑なに言ってたけど、八割くらいそうだと思う。	2009-11-04 13:48:51
5414641671	a_d_m_a_n	バズマン、今日の進捗全部メールしといてね。	2009-11-04 15:59:45
5436845684	a_d_m_a_n	まぢで？RT伊藤直樹がGTを卒業し、wieden+kenedyの東京オフィス代表に就任しました。	2009-11-05 10:26:18
5454125603	a_d_m_a_n	いろいろ、悩むなぁ。	2009-11-06 02:00:42
5453114308	a_d_m_a_n	デスクの上の本を整理しはじめて、早2時間。。。	2

### 前処理

In [50]:
import re
import pandas as pd

# 正規表現パターンと対応する置換表現を辞書で定義
replacements = {
    r"@(\S*)": "[USR]",
    r"http(\S*)": "[URL]",
}

# 文中のマッチした箇所を対応する置換表現に置換する関数
def apply_regex(text):
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)
    return text

# テキストファイルを読み込み、1つ目と3つ目のフィールドに前処理を適用し、それらを出力する関数
def preprocess_file(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        processed_data = []
        for line in file:
            fields = line.strip().split('\t')
            # 1つ目のフィールド（前処理なし）と3つ目のフィールド（前処理あり）を取得
            first_field = fields[0]
            third_field = apply_regex(fields[2])
            processed_data.append([first_field, third_field])
    return processed_data

In [51]:
# 関数を使用してファイルを前処理し、結果をDataFrameに格納
input_file = file_name  # 入力ファイル名
processed_data = preprocess_file(input_file)

# DataFrameを作成
df = pd.DataFrame(processed_data, columns=[['id', 'text']])

In [52]:
df

,id,text
0,5388869666,ディズニーランドホテルなう。
1,5389700374,舞浜地ビールなう。奈緒ちゃんの粋な計らい。
2,5394370590,ディズニーランドホテルは仕掛けがいろいろあって面白い。喫煙所が一つしかないけど…
3,5394459718,喫煙所は完全に隔離され、ちゃちじゃないがホテル全体のトンマナに悪影響を及ぼさない設計
4,5397779328,レヴィ＝ストロース氏死去､ 残念だ。ご冥福を祈ります。
...,...,...
6882,10998549309,せきがはらああああｲﾏｺｺ L:岐阜県関ケ原町関ケ原町
6883,11011605637,二条城でた！祇園に向かうよ
6884,11011830998,さむいまじ寒い春みたいなウキウキな服装しちゃってきたから寒い
6885,11021943880,伏見稲荷へ移動～ 八坂神社行った！よーじやで買い物したりかなり王道コースだよ 恋みくじは凶だ...


In [10]:
df.head()

,id,text
0,5367044072,ゼミのＯＢ会……だと……？ 全員参加だし……。修羅場の11月末の土曜日に何たる
1,5367066197,しかも会場が南国酒家って費用相当高いんじゃね？とか思ったら６０００円とかうわああああ
2,5367115249,値段からして結構良さげなコースだろうから楽しみではあるけど、この出費はちょっと大きいよ……
3,5367199541,うーんうーん……全員参加と銘打ってある以上、行かないとまずいよねぇ……
4,5367230602,原宿の南国酒家か……。一回行ったことあるなぁ。綺麗なところだった


In [55]:
df.shape

(6887, 2)

In [19]:
# 仮定: dfは既にあるデータフレーム
for index, row in df.iterrows():
    try:
        # 一時的なExcelファイルに行を保存してみる
        pd.DataFrame([row]).to_excel("temp.xlsx", index=False)
    except Exception as e:
        # エラーが発生した行のインデックスを表示
        print(f"エラーが発生した行: {index}")
        print(f"エラーの詳細: {e}")
        break  # 最初のエラーが見つかったらループを終了

# 一時ファイルの削除
import os
if os.path.exists("temp.xlsx"):
    os.remove("temp.xlsx")


エラーが発生した行: 2579
エラーの詳細: $B$5$h$&$J$i%(%C%U%'%kEc!"$3$s$K(B$B$A$O!D!D$3$l$I$3! cannot be used in worksheets.


In [30]:
df.iloc[2570:2610,1]

2570                                      あと10分で着くよー(^U^)
2571                                  倍額フラグビンビンなんですがどうしよう
2572                                       着いた！　待っててメリー達！
2573                                        蓮子の唾液ピッチャーで！！
2574                                 酔いはとうに醒めてるけど足腰が痛いお……
2575                                         着せ替え楽しすぎてつらい
2576                          7と10が鉄板……しかし個人的に8が可愛すぎるうううう
2577                                         いや待て、9もいいな……
2578                                     えらい雪積もってて興奮してるなう
2579    $B$5$h$&$J$i%(%C%U%'%kEc!"$3$s$K(B$B$A$O!D!...
2580            家の裏手が広い空き地みたいなところになってるから雪が降ると雪原みたくなって興奮する
2581                        早速玄関で盛大に転んできたよ。いたたたた……右手小指もげる
2582                                        すももさんの優しさに泣いた
2583                                   例大祭のめりさんは咲夜さんコスか……
2584                                     指が脱皮してるなう（定期ポスト）
2585                          エイリア学園に入ってガゼルとバーンと一緒にサッカーする
2586                                    すももなのだー（バズらせようず！）
2587          

In [56]:
# DataFrameをExcelファイルとして保存
# ファイル数が大きいとだめ
output_file = f'{file_name}.xlsx'  # 出力ファイルのパス
df.to_excel(output_file, index=True)